In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys
import data
import plotly.express as px
import pandas   as pd


In [ ]:
path = r"C:\Users\lnae0002\Desktop\autolamella\autolamella\log\3"

df_sample, df_history, df_shift, df_steps  = data.calculate_statistics_dataframe(path)
display(df_shift)

In [ ]:
points = pd.DataFrame(df_shift["shift"].tolist(), columns=["x", "y"])
points["lamella"] = df_shift["lamella"]
fig = px.scatter(points, x="x", y='y', color="lamella")
fig.show()

In [ ]:
display(df_steps)
fig = px.scatter(df_steps, x="step_n", y="timestamp", color="stage", symbol='lamella')
fig.show()



In [ ]:
# calculate difference in timestamp between rows
df_steps['delta'] = df_steps['timestamp'].diff()

display(df_steps)

px.bar(df_steps, x="lamella", y="delta", color="step", facet_col="stage")

In [ ]:
fig = px.bar(df_history, x="stage", y="duration", color="petname", barmode="group")

fig.show()

In [ ]:
# display(df_history)
df_history["duration"].mean()

df_sorted = df_history.groupby(["stage", "petname"]).mean()
df_sorted.drop(columns=["start", "end"], inplace=True)
df_sorted.reset_index(inplace=True)
display(df_sorted)


In [ ]:
points = pd.DataFrame(df_sample["lamella.centre"].tolist(), columns=["x", "y"])
points["petname"] = df_history["petname"].unique()
display(points)
fig = px.scatter(points, x="x", y='y', color="petname")
fig.show()

In [ ]:
df_sample["petname"] = df_history["petname"].unique()
fig = px.scatter_3d(df_sample, x="lamella.x", y='lamella.y', z='lamella.z', color="petname")
fig.show()

### SpaceTOMO

In [ ]:

from PIL import Image
import tifffile as tff
import glob



PATH = "../../../data/spacetomo/imagesTr/"


filenames = glob.glob(PATH + "*.png")

from pprint import pprint

# pprint(filenames)

# convert to tiff, save in separate dir   
for filename in filenames:
    im = Image.open(filename)
    im.save(filename.replace(".png", ".tif"))


### Feature Detection Tools



In [ ]:
%load_ext autoreload
%autoreload 2

import os
import glob
import pandas as pd



path = "../../../data/autolamella-paper/model-development/train/serial-liftout/test/"

filenames = sorted(glob.glob(os.path.join(path, "*.tif*")))


print(df.head())

In [ ]:
df = pd.read_csv(os.path.join(path, "data.csv"))

display(df.head())

filenames = []
for fname in df["filename"].values:
    filenames += glob.glob(os.path.join(path, f"*{fname}*"))

print(len(filenames))

from pprint import pprint
pprint(filenames)


In [ ]:
df = pd.read_csv(os.path.join(path, "data.csv"))


fname = df["filename"].values[0]

df_filt = df[df["filename"] == fname]


display(df)

# drop rows with filename = fname
df.drop(df[df["filename"] == fname].index, inplace=True)

display(df)

### Get / Set API

In [ ]:
%load_ext autoreload
%autoreload 2


from fibsem import utils, acquire
from fibsem.structures import BeamType
import matplotlib.pyplot as plt

import matplotlib
%matplotlib inline

microscope, settings = utils.setup_session(debug=True)

print('--'*50)
microscope.apply_configuration()

print('--'*50)
eb = microscope.get_beam_system_settings(BeamType.ELECTRON)
ib = microscope.get_beam_system_settings(BeamType.ION)



### Export Milling Protocol

In [ ]:
%load_ext autoreload
%autoreload 2


from fibsem.structures import BeamType
from fibsem import utils, patterning

microscope, settings = utils.setup_session(protocol_path="../../autolamella/protocol/protocol-serial-liftout.yaml", debug=True)

In [ ]:
stages = patterning.get_milling_stages("trench", settings.protocol["milling"])

In [ ]:

from pprint import pprint

protocol = patterning.get_protocol_from_stages(stages)
pprint(protocol)


# select file

# load protocol

# select key

# load stages / save stages


## Depth Anything



In [ ]:
from transformers import pipeline
from datasets import load_dataset
from PIL import Image
import requests
import numpy as np

# load pipe
pipe = pipeline(task = "depth-estimation", model="nielsr/depth-anything-small", device=0)

# load image
url = 'http://images.cocodataset.org/val2017/000000039769.jpg'
image = Image.open(requests.get(url, stream=True).raw)

ds = load_dataset("patrickcleeve/autolamella", "serial-liftout", split="test")

import random
idx = random.randint(0, len(ds)-1)
image = Image.fromarray(np.asarray(ds[idx]["image"]))
mask = np.asarray(ds[idx]["annotation"])

from fibsem.segmentation.utils import decode_segmap_v2

# inference
depth = pipe(image)["depth"]

import matplotlib.pyplot as plt
fig, ax = plt.subplots(1,2, figsize=(10,10))

ax[0].imshow(image, cmap="gray")
ax[0].imshow(decode_segmap_v2(mask), alpha=0.3)
ax[0].set_title("image + mask")
# ax[1].imshow(image, cmap="gray")
ax[1].imshow(depth, alpha=0.7, cmap="turbo")
ax[1].set_title("depth")

plt.show()

prin

In [ ]:
import numpy as np
print(np.min(depth), np.max(depth))

## Wedge Milling

- Tilt 1deg
- mill top rect
- Tilt back
- Tilt -1deg
- mill bottom rect



In [ ]:
%load_ext autoreload
%autoreload 2


from fibsem import utils, acquire, alignment
from fibsem.structures import BeamType
import matplotlib.pyplot as plt
from fibsem.patterning import get_milling_stages
from fibsem.structures import FibsemStagePosition


microscope, settings = utils.setup_session(protocol_path="/home/patrick/github/autolamella/autolamella/protocol/protocol-on-grid.yaml", debug=True)





In [ ]:
stages = get_milling_stages("lamella", settings.protocol["milling"])



from pprint import pprint

from fibsem.structures import ImageSettings, FibsemImage, FibsemRectangle
from fibsem.patterning import FibsemMillingStage, TrenchPattern
from fibsem import milling, patterning, alignment
from fibsem.microscope import FibsemMicroscope

# for stage in stages[:2]:

settings.image.beam_type = BeamType.ION
settings.image.reduced_area = FibsemRectangle(0.25, 0.25, 0.5, 0.5)
ref_image = acquire.acquire_image(microscope, settings.image)


In [ ]:

# milling.mill_stages_ui(microscope, stages)
for stage in stages[:2]:
    milling.mill_stage_with_overtilt(microscope, stage, ref_image, asynch=False)


### Segmentation Quality

Analysis of segmentation quality

In [ ]:
%load_ext autoreload
%autoreload 2
from fibsem.segmentation.model import load_model
from datasets import load_dataset

model = load_model("autolamella-liftout-20240107.pt")


In [ ]:
ds = load_dataset("patrickcleeve/autolamella", "liftout", split="test")

In [ ]:

import matplotlib.pyplot as plt
from fibsem.segmentation.utils import decode_segmap_v2
import numpy as np
import random


for i in range(1):
    # idx = random.randint(0, len(ds)-1)
    image = np.asarray(ds[idx]["image"])

    mask = np.asarray(ds[idx]["annotation"])

    # inference
    pred, scores = model.inference_v2(image, rgb=False)

    fig, ax = plt.subplots(1,2, figsize=(10,10))

    ax[0].imshow(image, cmap="gray")
    ax[0].imshow(decode_segmap_v2(mask), alpha=0.3)
    ax[0].set_title("image + mask")
    ax[1].imshow(image, cmap="gray")
    ax[1].imshow(decode_segmap_v2(pred[0]), alpha=0.3)
    ax[1].set_title("image + pred")

    plt.show()

    scores = scores.detach().cpu().numpy()

    # # plot scores for each class
    # fig, ax = plt.subplots(1,scores.shape[1], figsize=(20,5))

    # for i in range(scores.shape[1]):
    #     ax[i].imshow(scores[0,i], cmap="turbo")
    #     ax[i].set_title(f"class {i}")
    #     # colorbar
    #     cbar = plt.colorbar(ax[i].imshow(scores[0,i], cmap="turbo"), ax=ax[i])
    #     cbar.set_label('score')
        

    # plt.show()

    # get the max
    max_score = np.max(scores, axis=1)
    plt.imshow(max_score[0], cmap="turbo")
    plt.colorbar(plt.imshow(max_score[0], cmap="viridis"))
    plt.show()


    # get the unique values in the mask
    unique = np.unique(mask)
    print(unique)

    for u in unique:
        print(f"class {u}:", np.sum(mask == u))

        # get hte mask for the class
        labeled_mask = mask == u
        predicted_mask = pred[0] == u

        # calculate accuracy
        accuracy = (predicted_mask == labeled_mask)
        print(f"accuracy: {accuracy}")

        fig, ax = plt.subplots(1,3, figsize=(10,10))
        ax[0].imshow(labeled_mask, cmap="gray")
        ax[1].imshow(predicted_mask, cmap="gray")
        ax[2].imshow(accuracy, cmap="gray")

        plt.show()


        # calculate iou


        intersection = (predicted_mask & labeled_mask)
        union = (predicted_mask | labeled_mask)
        iou = (intersection / union)
        # print(f"iou: {iou}")
        plt.imshow(iou)
        plt.show()
        print(f"iou: {np.mean(iou)}")



        # intersection = (predicted_mask & labeled_mask).float().sum()
        # dice_coefficient = (2. * intersection) / (predicted_mask.sum() + labeled_mask.sum())

        # print(f"dice_coefficient: {dice_coefficient}")

        # import torch.nn.functional as F
        # loss = F.binary_cross_entropy(predicted_mask, labeled_mask)
        # print(f"loss: {loss}")



    # plot, mask, pred
    fig, ax = plt.subplots(1,3, figsize=(15,5))

    ax[0].imshow(mask[0], cmap="gray")
    ax[0].set_title("mask")
    ax[1].imshow(pred[0], cmap="gray")
    ax[1].set_title("pred")


    plt.show()



## Serial Liftout - Landing 

In [ ]:
%load_ext autoreload
%autoreload 2


from fibsem import utils
from autolamella.workflows import serial



from fibsem.structures import BeamType, FibsemImage


image1 = FibsemImage.load("data/00277.tif")
image2 = FibsemImage.load("data/00324.tif")
image3 = FibsemImage.load("data/00350.tif")


# rotate images by 180

import numpy as np

# image1.data = np.rot90(image1.data, 2)
# image2.data = np.rot90(image2.data, 2)
# image3.data = np.rot90(image3.data, 2)


import matplotlib.pyplot as plt

fig, ax = plt.subplots(1,3, figsize=(15,5))

ax[0].imshow(image1.data, cmap="gray")
ax[1].imshow(image2.data, cmap="gray")
ax[2].imshow(image3.data, cmap="gray")

plt.show()



In [ ]:
IMAGE_PATH = "/home/patrick/github/data/autolamella-paper/model-development/train/serial-liftout/train"
LABEL_PATH = "/home/patrick/github/data/autolamella-paper/model-development/train/serial-liftout/train/labels"

FNAME = "00324.tif"
FNAME = "00277.tif"
FNAME = "00350.tif"
import os

image = FibsemImage.load(os.path.join(IMAGE_PATH, FNAME))
label = FibsemImage.load(os.path.join(LABEL_PATH, FNAME))

import matplotlib.pyplot as plt
from fibsem.segmentation.utils import decode_segmap_v2

fig, ax = plt.subplots(1,2, figsize=(10,5))

ax[0].imshow(image.data, cmap="gray")
ax[1].imshow(decode_segmap_v2(label.data), cmap="gray")




In [ ]:
# split the image into half, horizontally

from fibsem.detection.detection import (detect_features, plot_detection, 
LandingGridLeftEdge, LandingGridRightEdge, 
VolumeBlockBottomLeftCorner, VolumeBlockBottomRightCorner, VolumeBlockTopRightCorner)
from fibsem.structures import Point


FNAMES  = ["00324.tif", "00277.tif", "00350.tif"]

from fibsem.segmentation.model import load_model
model = load_model("autolamella-serial-liftout-20240107.pt")

for fname in FNAMES:

    image = FibsemImage.load(os.path.join(IMAGE_PATH, fname))
    label = FibsemImage.load(os.path.join(LABEL_PATH, fname))
    mask = label.data
    # plt.imshow(decode_segmap_v2(mask))
    # plt.show()

    # inference
    det = detect_features(
        image=image,
        model=model,
        features=[
            VolumeBlockBottomLeftCorner(), VolumeBlockBottomRightCorner(), 
            LandingGridLeftEdge(), LandingGridRightEdge()]
    )

    plot_detection(det)

    # det = detect_features_v2(img = None, mask=mask, features=[LandingGridLeftEdge(), LandingGridRightEdge()])


    # # plot
    # plt.imshow(image.data, cmap="gray")
    # plt.imshow(decode_segmap_v2(mask), alpha=0.3)
    # for d in det:
    #     plt.plot(d.px.x, d.px.y, 'ro')
    # plt.show()


